## Predicting Titanic Survivors

As tragic as it was, Titanic has great value for any Machine Learning enthusiast - this is a great classification exercise.

Let's start with the plan.

### The Plan
* #### Phase 1: Baseline model
  1. Examine the data, start with already numeric values
  2. Fill in `na`s with medians, ignore if features that can't be transformed to numeric
  3. Test different algorithms
  4. Evaluate metrics
* #### Phase 2: Feature engineering
  1. Do a pass on each feature, see if they can be relevant
  2. Normalize features
  3. Train and compare metrics
  4. Declare the best model to work with
* #### Phase 3: Hyperparameter tuning

### Phase 1: Baseline model

#### 1. Examine the data, start with already numeric numbers

In [239]:
import pandas as pd

df = pd.read_csv("data/train.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


* 🎯 `Survived` is the target.
<hr />

* ❌ Skipping `PassengerId` as it seems redundant.
* ✅ Include `Pclass`, should have a direct impact. It's also already numerical.
* ❌ Skip `Name`, should be irrelevant.
* ❕ Skip `Sex` for now, will need transforming.
* ✅ Include `Age`, already numerical.
* ✅ Include `SibSp`, already numerical.
* ✅ Include `Parch`, already numerical.
* ❕ Skip `Ticket`, worth digging later on.
* ✅ Include `Fare`, already numerical.
* ❕ Skip `Cabin`, worth digging later on.
* ❕ Skip `Embarked`, worth digging later on.

So, out of 12 columns, we'll start with 4+1 (_`Survived` is target_).
Let's make a `baseline_df`

In [240]:
baseline_df = df[["Pclass", "Age", "SibSp", "Parch", "Fare", "Survived"]]
baseline_df

,Pclass,Age,SibSp,Parch,Fare,Survived
0,3,22.0,1,0,7.2500,0
1,1,38.0,1,0,71.2833,1
2,3,26.0,0,0,7.9250,1
3,1,35.0,1,0,53.1000,1
4,3,35.0,0,0,8.0500,0
...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,0
887,1,19.0,0,0,30.0000,1
888,3,NaN,1,2,23.4500,0
889,1,26.0,0,0,30.0000,1


#### 2. Fill in `na`s with medians

Let's check for `na`s.

In [241]:
baseline_df.isna().sum()

Pclass        0
Age         177
SibSp         0
Parch         0
Fare          0
Survived      0
dtype: int64

For `Age`, we'll just take the `median`.

In [242]:
baseline_df.loc[:, "Age"] = baseline_df["Age"].fillna(baseline_df["Age"].mean())
baseline_df.isna().sum()

Pclass      0
Age         0
SibSp       0
Parch       0
Fare        0
Survived    0
dtype: int64

 #### 3. Test different algorithms

We are good to go!
Now, let's see our contenders for classification:
* Logistic Regression
* Decision Tree Classifier
* Random Forest Classifier
* Support Vector Machine (SVM)
* K-Nearest Neighbors (KNN)
* Naive Bayes

In [243]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb

features = ["Pclass", "Age", "SibSp", "Parch", "Fare"]
target = "Survived"

X = baseline_df[features]
y = baseline_df[target]

classifiers = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Support Vector Machine": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
    "XGBoost": xgb.XGBClassifier(random_state=42),
}

Now that we made our cross validated predictions, let's see how they did with defaults and no tuning.

This should give us a good baseline to begin with.

####  4. Evaluate metrics

In [244]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


def evaluate(y, y_pred):
    accuracy = accuracy_score(y, y_pred)
    classification_rep = classification_report(y, y_pred, output_dict=True)
    confusion_mat = confusion_matrix(y, y_pred)

    return {
        "Accuracy": accuracy,
        "Precision (Not Survived)": classification_rep["0"]["precision"],
        "Recall (Not Survived)": classification_rep["0"]["recall"],
        "F1-Score (Not Survived)": classification_rep["0"]["f1-score"],
        "Precision (Survived)": classification_rep["1"]["precision"],
        "Recall (Survived)": classification_rep["1"]["recall"],
        "F1-Score (Survived)": classification_rep["1"]["f1-score"],
        "Confusion Matrix": confusion_mat,
    }

In [245]:
from sklearn.model_selection import cross_val_predict

import numpy as np

np.random.seed(42)

evaluation_results = {}

for name, clf in classifiers.items():
    y_pred = cross_val_predict(clf, X, y, cv=5)
    evaluation_results[name] = evaluate(y, y_pred)

# Create a DataFrame for evaluation results
evaluation_df = pd.DataFrame(evaluation_results).T
evaluation_df = evaluation_df.sort_values(by="Accuracy", ascending=False)
evaluation_df

,Accuracy,Precision (Not Survived),Recall (Not Survived),F1-Score (Not Survived),Precision (Survived),Recall (Survived),F1-Score (Survived),Confusion Matrix
Logistic Regression,0.694725,0.709531,0.854281,0.775207,0.652174,0.438596,0.524476,"[[469, 80], [192, 150]]"
XGBoost,0.684624,0.727119,0.781421,0.753292,0.601329,0.52924,0.562986,"[[429, 120], [161, 181]]"
Naive Bayes,0.680135,0.7,0.84153,0.764268,0.623377,0.421053,0.502618,"[[462, 87], [198, 144]]"
Random Forest,0.674523,0.729204,0.750455,0.739677,0.579755,0.552632,0.565868,"[[412, 137], [153, 189]]"
Support Vector Machine,0.673401,0.673854,0.910747,0.774593,0.671141,0.292398,0.407332,"[[500, 49], [242, 100]]"
K-Nearest Neighbors,0.652076,0.707826,0.741348,0.724199,0.550633,0.508772,0.528875,"[[407, 142], [168, 174]]"
Decision Tree,0.646465,0.704545,0.734062,0.719001,0.54232,0.505848,0.523449,"[[403, 146], [169, 173]]"


### Phase 2: Feature engineering

#### 1. Do a pass on each feature, see if they can be relevant

Let's review our data again:

In [246]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


Recall what features we had before:

> * 🎯 `Survived` is the target.
><hr />
>
> * ❌ Skipping `PassengerId` as it seems redundant.
> * ✅ Include `Pclass`, should have a direct impact. It's also already numerical.
> * ❌ Skip `Name`, should be irrelevant.
> * ❕ Skip `Sex` for now, will need transforming.
> * ✅ Include `Age`, already numerical.
> * ✅ Include `SibSp`, already numerical.
> * ✅ Include `Parch`, already numerical.
> * ❕ Skip `Ticket`, worth digging later on.
> * ✅ Include `Fare`, already numerical.
> * ❕ Skip `Cabin`, worth digging later on.
> * ❕ Skip `Embarked`, worth digging later on.

We cannot add or obtain new data, but we can definitely make use of more features and some transformations.

Redundant:
* ❌ `PassengerId`
* ❌ `Name`

Transform:
* ✅ `Sex`: One-hot encode
* ✅ `Embarked`: One-hot encode 
* ✅ `Age`: Min max scaler, for `na`s, we'll polyfill them based on `Sex`'s median
* ✅ `SibSp`: Min max scaler
* ✅ `Parch`: Min max scaler
* ✅ `Fare`: Log

New:
* ❕`Ticket`: There are two types of tickets, we can add a new `IsTicketNumeric` feature from this.
  * Alphanumeric: "CA. 2343", "PC 17612"
  * Numeric: "3101295", "1601"
* ❕`Cabin`: It might be difficult to analyze the impact of the cabin number without exactly knowing where it is located on the ship. In addition to this, we already have so many rows without any Cabin information (*687*). So we can just derive `IsCabinKnown` feature from this to see if this information helps.
* ❕`Embarked`: This feature may hold interesting information, we can just one-hot encode this. For `na` data, we'll just assume it's the most common one (*S*).

Seems fine:
* ✅ `Pclass`

#### 2. Normalize features

This imputer will take medians per gender, instead of taking median age for all passengers.

In [247]:
from sklearn.base import BaseEstimator, TransformerMixin



class MedianAgeImputer(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):

        self.median_ages_ = X.groupby("Sex")["Age"].median()

        return self


    def transform(self, X, y=None):

        X = X.copy()

        for sex in self.median_ages_.index:

            X.loc[(X["Age"].isnull()) & (X["Sex"] == sex), "Age"] = self.median_ages_[
                sex
            ]

        return X

In [248]:
from sklearn.preprocessing import MinMaxScaler


def preprocess_titanic_data(df):
    # Drop PassengerId and Name
    df_improved = df.drop(["PassengerId", "Name"], axis=1)

    # Impute Age
    imputer = MedianAgeImputer()
    df_improved = imputer.fit_transform(df_improved)

    # One-hot encode Sex feature
    df_improved["IsMale"] = df_improved.Sex.map({"male": 1, "female": 0})
    df_improved.drop("Sex", axis=1, inplace=True)

    # One-hot encode Embarked feature
    df_improved["Embarked"] = df_improved.Embarked.fillna("S")
    df_improved = pd.get_dummies(df_improved, columns=["Embarked"], drop_first=True)
    df_improved["Embarked_Q"] = df_improved.Embarked_Q.map({True: 1, False: 0})
    df_improved["Embarked_S"] = df_improved.Embarked_S.map({True: 1, False: 0})

    # MinMaxScaler for Age, Pclass, SibSp, Parch
    scaler = MinMaxScaler()
    df_improved[["Age", "Pclass", "SibSp", "Parch"]] = scaler.fit_transform(
        df_improved[["Age", "Pclass", "SibSp", "Parch"]]
    )

    # Take log of Fare
    df_improved["Fare"] = df_improved["Fare"].map(lambda x: np.log(x) if x > 0 else 0)

    # Populate IsTicketNumeric feature
    df_improved["IsTicketNumeric"] = df_improved["Ticket"].str.isnumeric().astype(int)
    df_improved.drop(columns=["Ticket"], axis=1, inplace=True)

    # Populate IsCabinKnown feature
    df_improved["IsCabinKnown"] = df_improved["Cabin"].notna().astype(int)
    df_improved.drop(columns=["Cabin"], axis=1, inplace=True)

    return df_improved

#### 3. Train and compare metrics

In [249]:
np.random.seed(42)

improved_evaluation_results = {}

df_improved = preprocess_titanic_data(df.drop(["Survived"], axis=1))
X = df_improved
y = df.Survived

for name, clf in classifiers.items():
    y_pred = cross_val_predict(clf, X, y, cv=5)
    improved_evaluation_results[name] = evaluate(y, y_pred)

# Create a DataFrame for evaluation results
improved_evaluation_df = pd.DataFrame(improved_evaluation_results).T
improved_evaluation_df = improved_evaluation_df.sort_values(
    by="F1-Score (Survived)", ascending=False
)

display(evaluation_df)
display(improved_evaluation_df)

,Accuracy,Precision (Not Survived),Recall (Not Survived),F1-Score (Not Survived),Precision (Survived),Recall (Survived),F1-Score (Survived),Confusion Matrix
Logistic Regression,0.694725,0.709531,0.854281,0.775207,0.652174,0.438596,0.524476,"[[469, 80], [192, 150]]"
XGBoost,0.684624,0.727119,0.781421,0.753292,0.601329,0.52924,0.562986,"[[429, 120], [161, 181]]"
Naive Bayes,0.680135,0.7,0.84153,0.764268,0.623377,0.421053,0.502618,"[[462, 87], [198, 144]]"
Random Forest,0.674523,0.729204,0.750455,0.739677,0.579755,0.552632,0.565868,"[[412, 137], [153, 189]]"
Support Vector Machine,0.673401,0.673854,0.910747,0.774593,0.671141,0.292398,0.407332,"[[500, 49], [242, 100]]"
K-Nearest Neighbors,0.652076,0.707826,0.741348,0.724199,0.550633,0.508772,0.528875,"[[407, 142], [168, 174]]"
Decision Tree,0.646465,0.704545,0.734062,0.719001,0.54232,0.505848,0.523449,"[[403, 146], [169, 173]]"


,Accuracy,Precision (Not Survived),Recall (Not Survived),F1-Score (Not Survived),Precision (Survived),Recall (Survived),F1-Score (Survived),Confusion Matrix
XGBoost,0.824916,0.849023,0.870674,0.859712,0.783537,0.751462,0.767164,"[[478, 71], [85, 257]]"
Random Forest,0.809203,0.835398,0.859745,0.847397,0.763804,0.72807,0.745509,"[[472, 77], [93, 249]]"
K-Nearest Neighbors,0.799102,0.817869,0.867031,0.841733,0.763754,0.690058,0.725038,"[[476, 73], [106, 236]]"
Logistic Regression,0.790123,0.824373,0.837887,0.831075,0.732733,0.71345,0.722963,"[[460, 89], [98, 244]]"
Decision Tree,0.785634,0.829044,0.821494,0.825252,0.717579,0.72807,0.722787,"[[451, 98], [93, 249]]"
Support Vector Machine,0.787879,0.811419,0.854281,0.832298,0.744409,0.681287,0.71145,"[[469, 80], [109, 233]]"
Naive Bayes,0.762065,0.824663,0.779599,0.801498,0.674731,0.733918,0.703081,"[[428, 121], [91, 251]]"


#### 4. Declare the best model to work with

**XGBoost** seems to be the clear winner here.

However, Random Forest seems to be pretty close.

Let's think of ways to tune Random XGBoost further.

### Phase 3: Hyperparameter tuning

See if we can find better parameters than the defaults.

Let's remember the [default parameters](https://xgboost.readthedocs.io/en/stable/parameter.html) we had:
* `n_estimators`: 100 (from `sklearn`)
* `max_depth`: 6
* `learning_rate`: 0.3
* `subsample`: 1

In [250]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

default_xgb = xgb.XGBClassifier(random_state=42)
xgbs = {
    "Default": default_xgb,
    "Optimized": None,
}

np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

pipeline = Pipeline([("xgb", xgb.XGBClassifier(random_state=42))])

param_grid_xgb = {
    "xgb__n_estimators": [210, 215, 220],
    "xgb__max_depth": [3],
    "xgb__learning_rate": [0.012, 0.013, 0.014],
    "xgb__subsample": [0.5, 0.53, 0.57],
}

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid_xgb,
    cv=5,
    n_jobs=-1,
    verbose=2,
    scoring="accuracy",
    error_score="raise",
)
grid_search.fit(X_train, y_train)
xgbs["Optimized"] = grid_search.best_estimator_

grid_search.best_params_

Fitting 5 folds for each of 27 candidates, totalling 135 fits


{'xgb__learning_rate': 0.012,
 'xgb__max_depth': 3,
 'xgb__n_estimators': 220,
 'xgb__subsample': 0.5}

`GridSearchCV` thinks we are good with these hyperparameters.

Next, we'll get some metrics again to see how we did against our baseline model.

In [251]:
xgbs_evaluation_results = {}

for name, clf in xgbs.items():
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    y_cv_pred = cross_val_predict(clf, X, y, cv=5)

    xgbs_evaluation_results[name] = (
        classification_report(y_test, y_test_pred, output_dict=True),
        classification_report(y, y_cv_pred, output_dict=True),
    )


# Some formatting, so we can compare reports better
def html_classification_report(evaluation_results):
    def format_classification_report(report):
        report_df = pd.DataFrame.from_dict(report).transpose()
        report_df = report_df.round(2)

        if "support" in report_df.columns:
            report_df["support"] = report_df["support"].astype(int)

        return report_df.to_html()

    rows = ""
    for model_name, reports in evaluation_results.items():
        rows += f"""
        <tr>
            <td><h3>{model_name} Model Test Performance</h3>{format_classification_report(reports[0])}</td>
            <td><h3>{model_name} Model CV Performance</h3>{format_classification_report(reports[1])}</td>
        </tr>
        """

    html_content = f"""
    <table>
        {rows}
    </table>
    """
    return HTML(html_content)


from IPython.display import display, HTML

display(html_classification_report(xgbs_evaluation_results))

,precision,recall,f1-score,support
0,0.84,0.83,0.83,105
1,0.76,0.77,0.77,74
accuracy,0.80,0.80,0.80,0
macro avg,0.80,0.80,0.80,179
weighted avg,0.80,0.80,0.80,179
,precision,recall,f1-score,support
0,0.85,0.87,0.86,549
1,0.78,0.75,0.77,342
accuracy,0.82,0.82,0.82,0
macro avg,0.82,0.81,0.81,891


Questionable at best, tuning didn't really help much. 

We'll still go with the optimized model though.

### Phase 4: Predict survivors!

In [252]:
# Load the test dataset
test_data = pd.read_csv("data/test.csv")

# Preprocess the test dataset
X = preprocess_titanic_data(test_data)
y_pred = xgbs["Optimized"].predict(X)

submission = pd.DataFrame({"PassengerId": test_data["PassengerId"], "Survived": y_pred})
submission.to_csv("submission.csv", index=False)
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


Use `kaggle competitions submit -c titanic -f submission.csv -m "Message"` to submit!